<a href="https://colab.research.google.com/github/gcs1005/TFG--Bioinform-tica/blob/main/Script_TFG_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script TFG en R


En la primera celda de código escribimos lo siguiente para instalar IRkernel:

In [ ]:
install.packages('IRkernel')


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Ejecutamos la siguiente línea de código para habilitar el kernel de R en Colab

In [ ]:
IRkernel::installspec()

Comenzamos ahora con el script para la descarga de datos sanos para los genes de interés a partir de TCGA y usando TCGAbiolinks

Previamente debemos realizar la carga de los siguientes paquetes en R.



In [ ]:
library(tidyverse)

install.packages('pheatmap')
library(pheatmap)

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install()
BiocManager::install("SummarizedExperiment")
BiocManager::install("SummarizedExperiment", force = TRUE)
library(SummarizedExperiment)

BiocManager::install("TCGAbiolinks")
install.packages('TCGAbiolinks')
library(TCGAbiolinks)


BiocManager::install("maftools")
install.packages('maftools')
library(maftools)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()   masks base::%||%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.23), R 4.4.0 (2024-04-24)

Installin

# Para los casos sanos

## Primera solicitud(perfil del transcriptoma)

In [ ]:
#Y comenzaremos dando uso a una de las 3 funciones que vamos a utilizar:
#Una función que nos permitirá ver los proyectos presentes en GDC.

proyectos_gcd<-getGDCprojects()

#con el siguiente comando obtenemos un resumen por consola del proyecto de
#interés(aquel que incluye información del gen BRCA):
getProjectSummary('TCGA-BRCA')

#A continuación realizamos el filtrado del proyecto de TCGA-BRCA con la consulta
#siguiente.
query_TCGA<-GDCquery(project = 'TCGA-BRCA',
         data.category = 'Transcriptome Profiling')

#Y observamos los resultados obtenidos de la petición
output_query_TCGA<-getResults(query_TCGA)

#/*
#Primera solicitud (perfil del transcriptoma):
#Con los datos ya observados, hacemos un filtrado de aquellos que nos interesan:
query_TCGA_normal<-GDCquery(project = 'TCGA-BRCA',
                     data.category = 'Transcriptome Profiling',
                     experimental.strategy = 'RNA-Seq',
                     sample.type = 'Solid Tissue Normal',
                     access = 'open')

#Ahora solicitamos la visualización de los resultados obtenidos con la
#solicitud:

output_TCGA_normal<-getResults(query_TCGA_normal)

#Con ello, ya descargaríamos los datos:

GDCdownload(query_TCGA_normal)


,file_count,case_count,data_category
,<int>,<int>,<chr>
1,17337,1098,Simple Nucleotide Variation
2,9281,1098,Sequencing Reads
3,5316,1098,Biospecimen
4,2288,1098,Clinical
5,12292,1098,Copy Number Variation
6,4876,1097,Transcriptome Profiling
7,3714,1097,DNA Methylation
8,919,881,Proteome Profiling
9,226,101,Somatic Structural Variation


--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BRCA

--------------------

oo Filtering results

--------------------

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases


ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BRCA

--------------------

oo Filtering results

--------------------

ooo By access

ooo By experimental.strateg

Downloading: 120 MB     

In [ ]:
#Una vez hayamos descargados los datos, podríamos realizar la preparación de los
#datos para su análisis.
#lo que nos devolverá un objeto experimento resumido(ya que ponemos el argumento
#bandera como 'TRUE').
datos_transcriptoma_sanos<-GDCprepare(query_TCGA_normal,summarizedExperiment = TRUE)
matriz_transcriptoma_sanos<-assay(datos_transcriptoma_sanos,'stranded_second')

|====================================================|100%                      Completed after 11 s 


Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 

brca subtype information from:doi.org/10.1016/j.ccell.2018.03.014

Available assays in SummarizedExperiment : 
  => unstranded
  => stranded_first
  => stranded_second
  => tpm_unstrand
  => fpkm_unstrand
  => fpkm_uq_unstrand



## Segunda solicitud (Valor Beta de Metilación):

In [ ]:
#/*
#Segunda solicitud (Valor Beta de Metilación):
TCGA_methylation_sanos<-GDCquery(project = 'TCGA-BRCA',
                                 data.category = 'DNA Methylation',
                                 data.type = 'Methylation Beta Value',
                                 platform = 'Illumina Human Methylation 27',
                                 sample.type = 'Solid Tissue Normal',
                                 access = 'open')
#Ahora solicitamos la visualización de los resultados obtenidos con la
#solicitud:

output_TCGA_methylation_sanos<-getResults(TCGA_methylation_sanos)

#Con ello, ya descargaríamos los datos:

GDCdownload(TCGA_methylation_sanos)

#Una vez hayamos descargados los datos, podríamos realizar la preparación de los
#datos para su análisis.
#lo que nos devolverá un objeto experimento resumido(ya que ponemos el argumento
#bandera como 'TRUE').

#grafico que prueba las diferencias entre los distintos valores beta de
#los datos
BiocManager::install("sesameData")
install.packages("sesameData")

datos_metilacion_sanos<-GDCprepare(TCGA_methylation_sanos,summarizedExperiment = TRUE)
assay(datos_metilacion_sanos)

idx<- datos_metilacion_sanos %>%
  assay %>%
  rowVars() %>%
  order(decreasing = TRUE) %>%
  head(10)


pheatmap(assay(datos_metilacion_sanos)[idx,])

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BRCA

--------------------

oo Filtering results

--------------------

ooo By platform

ooo By access

ooo By data.type

ooo By sample.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------


GDCdownload will download 27 files. A total of 20.762313 MB




Downloading: 8.6 MB     

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.23), R 4.4.0 (2024-04-24)

Installing package(s) 'sesameData'

also installing the dependencies ‘ExperimentHub’, ‘AnnotationHub’


Old packages: 'bit', 'brio', 'bslib', 'callr', 'curl', 'data.table', 'DBI',
  'dbplyr', 'devtools', 'digest', 'fs', 'gargle', 'ggplot2', 'gh', 'gtable',
  'highr', 'htmltools', 'httr2', 'isoband', 'knitr', 'munsell', 'openssl',
  'pkgbuild', 'processx', 'ragg', 'remotes', 'rmarkdown', 'rstudioapi',
  'rvest', 'sass', 'systemfonts', 'testthat', 'textshaping', 'tidyselect',
  'tinytex', 'usethis', 'whisker', 'xfun', 'xopen', 'zip', 'codetools',
  'lattice', 'MASS', 'Matrix', 'nlme', 'survival'

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘sesameData’ is not avail

|====================================================|100%                      Completed after 1 s 


-------------------

oo Merging 27 files

-------------------

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

Creating a SummarizedExperiment from DNA methylation input



ERROR: Error: sesame package is needed for this function to work. Please install it.


## Tercera solicitud (Variación simple de nucleótidos):

In [ ]:
#/*
#Tercera solicitud (Variación simple de nucleótidos):
TCGA_variation_sanos<-GDCquery(project = 'TCGA-BRCA',
                               data.category = 'Simple Nucleotide Variation',
                               data.type = 'Masked Somatic Mutation',
                               sample.type = 'Solid Tissue Normal',
                               access = 'open')
#Ahora solicitamos la visualización de los resultados obtenidos con la
#solicitud:

output_TCGA_variation_sanos<-getResults(query_TCGA_variation)

#Con ello, ya descargaríamos los datos:

GDCdownload(TCGA_variation_sanos)

#Una vez hayamos descargados los datos, podríamos realizar la preparación de los
#datos para su análisis.
#lo que nos devolverá un objeto experimento resumido(ya que ponemos el argumento
#bandera como 'TRUE').
datos_variacion_sanos<- GDCprepare(query_TCGA_variation,summarizedExperiment = TRUE)

#visualización de gráficos relativos a la variación de nucleótidos
 maftools.input<- read.maf(datos_variacion_sanos)

 plotmafSummary(maf= maftools.input,
                addStat = 'median',
                rmOutlier = TRUE,
                dashboard = TRUE)

 #oncoprint

 oncoplot(maf = maftools.input,
          top = 10,
          removeNonMutated = TRUE)




--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-BRCA

--------------------

oo Filtering results

--------------------

ooo By access

ooo By data.type





|sample.type | Number of samples|
|:-----------|-----------------:|
|            |                83|


ERROR: Error in GDCquery(project = "TCGA-BRCA", data.category = "Simple Nucleotide Variation", : Please set a valid sample.type argument from the list above.


# Para los casos patológicos

## Primera solicitud (perfil del transcriptoma):

In [ ]:
#Primera solicitud (perfil del transcriptoma):
 query_TCGA_enfermos<-GDCquery(project = 'TCGA-BRCA',
                             data.category = 'Transcriptome Profiling',
                             experimental.strategy = 'RNA-Seq',
                             sample.type = 'Primary Tumor',
                             access = 'open')



 output_TCGA_enfermos<-getResults(query_TCGA_enfermos)


 GDCdownload(query_TCGA_enfermos)

In [ ]:

 #Una vez hayamos descargados los datos, podríamos realizar la preparación de los
 #datos para su análisis.
 #lo que nos devolverá un objeto experimento resumido(ya que ponemos el argumento
 #bandera como 'TRUE').
 datos_transcriptoma_enfermos<- GDCprepare(query_TCGA_enfermos,summarizedExperiment = TRUE)
 matriz_transcriptoma_enfermos<-assay(datos_transcriptoma_enfermos,'unstranded')

## Segunda solicitud (Valor Beta de Metilación):

In [ ]:
#Segunda solicitud (Valor Beta de Metilación):
 TCGA_methylation_enfermos<-GDCquery(project = 'TCGA-BRCA',
                                  data.category = 'DNA Methylation',
                                  data.type = 'Methylation Beta Value',
                                  plataform = 'Illumina Human Methylation 27',
                                  sample.type = 'Primary Tumor',
                                  access = 'open')
 #Ahora solicitamos la visualización de los resultados obtenidos con la
 #solicitud:

 output_TCGA_methylation_enfermos<-getResults(TCGA_methylation_enfermos)

 #Con ello, ya descargaríamos los datos:

 GDCdownload(TCGA_methylation_enfermos)

 #Una vez hayamos descargados los datos, podríamos realizar la preparación de los
 #datos para su análisis.
 #lo que nos devolverá un objeto experimento resumido(ya que ponemos el argumento
 #bandera como 'TRUE').

 #grafico que prueba las diferencias entre los distintos valores beta de
 #los datos
 datos_metilacion_enfermos<- GDCprepare(TCGA_methylation_enfermos,summarizedExperiment = TRUE)
 assay(datos_metilacion_enfermos)

 idx<- datos_metilacion_enfermos %>%
   assay %>%
   rowVars() %>%
   order(decreasing = TRUE) %>%
   head(10)


 pheatmap(assay(datos_metilacion_enfermos)[idx,])

## Tercera solicitud (Variación simple de nucleótidos):

In [ ]:
#Tercera solicitud (Variación simple de nucleótidos):
 TCGA_variation_enfermos<-GDCquery(project = 'TCGA-BRCA',
                                data.category = 'Simple Nucleotide Variation',
                                data.type = 'Masked Somatic Mutation',
                                sample.type = 'Primary Tumor',
                                access = 'open')
 #Ahora solicitamos la visualización de los resultados obtenidos con la
 #solicitud:

 output_TCGA_variation_enfermos<-getResults(TCGA_variation_enfermos)

 #Con ello, ya descargaríamos los datos:

 GDCdownload(TCGA_variation_enfermos)

 #Una vez hayamos descargados los datos, podríamos realizar la preparación de los
 #datos para su análisis.
 #lo que nos devolverá un objeto experimento resumido(ya que ponemos el argumento
 #bandera como 'TRUE').
 datos_variacion_enfermos<- GDCprepare(TCGA_variation_enfermos,summarizedExperiment = TRUE)

 #visualización de gráficos relativos a la variación de nucleótidos
 maftools.input<- read.maf(datos_variacion_enfermos)

 plotmafSummary(maf= maftools.input,
                addStat = 'median',
                rmOutlier = TRUE,
                dashboard = TRUE)

 #oncoprint

 oncoplot(maf = maftools.input,
          top = 10,
          removeNonMutated = TRUE)
